
<font color="#1874CD"><h1 align="left">**Datamecum**</h1></font>
<font color="#6E6E6E"><h2 align="left">**Árboles de decisión**</h2></font>
<font color="#6E6E6E"><h3 align="left">Notebook: aplicación de árboles de decisión en un problema de regresión</h3></font>
***

El objetivo de este *notebook* es aplicar el algoritmo de árboles de decisión en un problema de regresión. 

Concretamente, se considera un conjunto de datos para predecir el precio medio de una vivienda en función del resto de variables explicativas disponibles.

In [2]:
# ------------------------------------------------------------------------------
# Librerías
# ------------------------------------------------------------------------------
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text

## Para establecer otra estrategia de búsqueda de hiperparámetros
import scipy
from sklearn.model_selection import RandomizedSearchCV

## Regresión lineal
from sklearn.linear_model import LinearRegression

np.random.seed(19)

# ------------------------------------------------------------------------------
# Configuración warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings('once')

In [4]:
## Para visualizar todas las columnas
pd.options.display.max_columns = None

## Leemos los datos
df = pd.read_csv('data/housing_dataset.csv')
df.head(5)

LotFrontage  LotArea  LotShape  LandContour  LotConfig  LandSlope  \
0         65.0     8450         0            3          0          0   
1         80.0     9600         0            3          3          0   
2         68.0    11250         1            3          0          0   
3         84.0    14260         1            3          3          0   
4         85.0    14115         1            3          0          0   

   OverallQual  OverallCond  YearBuilt  YearRemodAdd  RoofMatl_CompShg  \
0            7            5       2003          2003                 1   
1            6            8       1976          1976                 1   
2            7            5       2001          2002                 1   
3            8            5       2000          2000                 1   
4            5            5       1993          1995                 1   

   MasVnrArea  ExterQual  ExterCond  BsmtQual  BsmtCond  BsmtExposure  \
0       196.0          4          3         4         3             1   
1         0.0          3          3         4         3             4   
2       162.0          4          3         4         3             2   
3       350.0          4          3         4         3             3   
4         0.0          3          3         4         3             1   

   BsmtFinType1  BsmtFinSF1  BsmtFinType2  BsmtFinSF2  BsmtUnfSF  TotalBsmtSF  \
0             6         706             1           0        150          856   
1             5         978             1           0        284         1262   
2             6         486             1           0        434          920   
3             6         655             1           0        490         1145   
4             6         732             1           0         64          796   

   Heating_GasA  HeatingQC  CentralAir  Electrical  1stFlrSF  2ndFlrSF  \
0             1          5           1           5       856       854   
1             1          5           1           5      1262         0   
2             1          5           1           5       920       866   
3             1          5           1           5      1145      1053   
4             1          5           1           5       796       566   

   LowQualFinSF  GrLivArea  BsmtFullBath  BsmtHalfBath  FullBath  HalfBath  \
0             0       1710             1             0         2         1   
1             0       1262             0             1         2         0   
2             0       1786             1             0         2         1   
3             0       2198             1             0         2         1   
4             0       1362             1             0         1         1   

   BedroomAbvGr  KitchenAbvGr  KitchenQual  TotRmsAbvGrd  Functional  \
0             3             1            4             8           8   
1             3             1            3             6           8   
2             3             1            4             6           8   
3             4             1            4             9           8   
4             1             1            3             5           8   

   Fireplaces  FireplaceQu  GarageType  GarageYrBlt_gt_1990  GarageFinish  \
0           0            0           5                    1             2   
1           1            3           5                    0             2   
2           1            3           5                    1             2   
3           1            3           5                    1             2   
4           0            0           5                    1             1   

   GarageCars  GarageArea  GarageQual  GarageCond  PavedDrive  WoodDeckSF  \
0           2         548           3           3           2           0   
1           2         460           3           3           2         298   
2           2         608           3           3           2           0   
3           3         836           3           3           2         192   
4   

In [ ]:
print(f"Número de observaciones y columnas: {df.shape}")

In [ ]:
## NAs
df.isna().sum().sum()

La variable a predecir es la que se llama **SalePrice**. El resto son variables explicativas del modelo.

**Destacar que estos datos ya están limpios, por lo que no tenemos que aplicar ningún preprocesado.**

## División del conjunto de datos

En este punto vamos a dividir el conjunto de datos en dos subconjuntos: entrenamiento y test. Como hemos comentado es importante cuantificar su capacidad predictiva con nuevas observaciones y así evitar el sobreajuste.

In [6]:
## División en entrenamiento y validación. Forma 1
X_train, X_test, y_train, y_test = train_test_split(
                                        df.drop(columns = "SalePrice"), ## X
                                        df['SalePrice'], ## y
                                        random_state = 3,
                                        test_size=0.3
                                    )

In [7]:
## División en entrenamiento y validación. Forma 2
df_train, df_test = train_test_split(df, test_size = 0.3, random_state = 3)

## Ahora dividimos en X e y
y_train = df_train["SalePrice"]
y_test  = df_test["SalePrice"]

X_train = df_train.drop("SalePrice", axis=1)
X_test  = df_test.drop("SalePrice", axis=1)

Vemos que la variable a predecir posee una distribución asimétrica positiva, es decir, la cola es más larga en valores superiores a la media.

El conjunto de test también sigue la misma distribución. Esto es importante, ya que el modelo al final aprende una determinada distribución de los datos.

In [ ]:
df_train["SalePrice"].hist(bins=20)
df_test["SalePrice"].hist(bins=20)

## Entrenamiento del modelo

La clase <a href="https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html">DecisionTreeRegressor</a> del módulo *sklearn.tree* permite entrenar árboles de decisión para problemas de regresión. 

Destacar los siguientes parámetros:

- **max_depth:** profundidad máxima del árbol.
- **min_samples_split:** número mínimo de observaciones que debe de tener un nodo para que pueda dividirse. Resaltar que si es entero, se refiere a número de observaciones y si es decimal, se refiere a la proporción del tota de observaciones de entrenamiento.
- **min_samples_leaf:** número mínimo de observaciones que debe de tener cada uno de los nodos hijos para que se produzca la división. Resaltar que si es entero, se refiere a número de observaciones y si es decimal, se refiere a la proporción del tota de observaciones de entrenamiento.
- **max_leaf_nodes:** número máximo de nodos terminales.

In [ ]:
## Creamos el modelo
modelo = DecisionTreeRegressor(max_depth = 4, min_samples_split=0.01)

## Entrenamiento del modelo
modelo.fit(X_train, y_train)

In [ ]:
## Dibujamos el árbol de decisión

print(f"Profundidad del árbol: {modelo.get_depth()}")
print(f"Número de nodos terminales: {modelo.get_n_leaves()}")

plt.figure(figsize=(20,10), dpi=150)

plot = plot_tree(
            decision_tree = modelo,
            feature_names = X_train.columns.to_list(),
            filled        = True,
            rounded       = True,
            proportion    = True,
            fontsize      = 5
       )

In [ ]:
texto_modelo = export_text(
                    decision_tree = modelo,
                    feature_names = list(df.drop(columns = "SalePrice").columns)
               )
print(texto_modelo)

Antes de continuar, hacemos una pequeña prueba: ¿cuántas predicciones únicas obtenemos? Debería coincidir con el número de segmentos (nodos hojas) que obtenemos.

In [ ]:
len(np.unique(modelo.predict(X_train)))

Si comparamos predicciones en entrenamiento, tenemos lo siguiente.

In [ ]:
modelo.predict(X_train[:5])

In [ ]:
y_train[:5]

Las predicciones no se alejan tanto del valor real, pero cuidado, estamos mirando en entrenamiento.

Mirando la métrica del coeficiente de determinación en entrenamiento y test, vemos que hay sobreajuste. Esto indica que deberíamos jugar con los hiperparámetros para encontrar una combinación que nos ayude a minimizar el sobreajuste.

In [ ]:
## Calidad predictiva del modelo. En regresión devuelve el R^2, coeficiente de determinación

print("Score training = {}".format(modelo.score(X_train, y_train)))
print("Score test = {}".format(modelo.score(X_test, y_test)))

## Hay sobreajuste!

En regresión, es interesante graficar el valor real frente al valor predicho para visualizar cómo de bien predice.

In [18]:
## Calculamos la predicciones para poder graficarlas
y_train_est = modelo.predict(X_train)
y_test_est  = modelo.predict(X_test)

In [ ]:
mi = min(y_train_est.min(), y_test_est.min(), y_train.min(), y_test.min())
ma = max(y_train_est.max(), y_test_est.max(), y_train.max(), y_test.max())
aux = [mi, ma]

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(y_train, y_train_est, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (training)")
plt.ylabel("Precio estimado (training)")

plt.subplot(1,2,2)
plt.plot(y_test, y_test_est, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (test)")
plt.ylabel("Precio estimado (test)");

Cuanto más cerca de la diagonal, mejor predice el modelo. Destacar que aquellos puntos por encima de la diagonal son precios sobreestimados, mientras que por debajo son precios subestimados o infravalorados.

Observamos que existen errores, ya que tenemos segmentos de probabilidad (son las horizontales que se observan).

Otra buena práctica es analizar el histograma de residuos (errores) en el conjunto de test, ya que así cuantificamos el error y podemos identificar sesgos.

In [ ]:
plt.hist(y_test - y_test_est, 40);

## Importancia de los predictores

En un modelo es importante conocer la importancia de los predictores, ya que nos ayudan a saber qué características son las que correlan con la variable que se quiere predecir. Además, es una herramienta que nos permite eliminar aquellas variables que no aporten información.

La importancia de cada predictor se calcula como la reducción total del criterio de división que consigue el predictor en las divisiones en las que participa. En este caso, el criterio de divisón es el *MSE*. Entonces, si un predictor no ha sido seleccionado en ninguna división, su importancia es 0.

In [ ]:
importancia_predictores = pd.DataFrame(
                            {'predictor': df.drop(columns = "SalePrice").columns,
                             'importancia': modelo.feature_importances_}
                            )

print("Importancia de los predictores en el modelo")
print("")

importancia_predictores.sort_values('importancia', ascending=False)

## Optimización de los hiperparámetros

In [ ]:
param_grid = {"max_leaf_nodes": range(2, 41),
              "max_depth": range(1, 10)}

np.random.seed(0)

grid = GridSearchCV(DecisionTreeRegressor(), 
                    param_grid, 
                    cv=10,
                    verbose=3, 
                    scoring='r2') ## Por defecto es r2

grid.fit(X_train, y_train)

In [ ]:
## Vemos cuál ha sido el mejor modelo y su R^2
print(f"Mejor score: \n{grid.best_score_}\n")
print(f"Mejor modelo: \n{grid.best_estimator_}")

In [26]:
## Guardamos el mejor modelo para usarlo
model = grid.best_estimator_

In [ ]:
## Volvemos a validar con entrenamiento y test
print("Score training = {}".format(model.score(X_train, y_train)))
print("Score test = {}".format(model.score(X_test, y_test)))

## Hay sobreajuste!

In [28]:
## Calculamos la predicciones para poder graficarlas
y_train_est = model.predict(X_train)
y_test_est  = model.predict(X_test)

In [ ]:
mi = min(y_train_est.min(), y_test_est.min(), y_train.min(), y_test.min())
ma = max(y_train_est.max(), y_test_est.max(), y_train.max(), y_test.max())
aux = [mi, ma]

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(y_train, y_train_est, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (training)")
plt.ylabel("Precio estimado (training)")

plt.subplot(1,2,2)
plt.plot(y_test, y_test_est, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (test)")
plt.ylabel("Precio estimado (test)");

Existen otras estrategias para optimizar los hiperparámetros. Por ejemplo, con *RandomizedSearchCV* realiza tantas iteraciones como nosotros queramos seleccionando de forma aleatoria hiperparámetros dentro de un rango dado.

In [ ]:
params = {"max_leaf_nodes": range(2,41),
          "max_depth": range(2,20)} ## 702 combinaciones

np.random.seed(0)

random_search = RandomizedSearchCV(DecisionTreeRegressor(), params, cv=10, n_iter=100, verbose=10) ## Restringir a 100 iteraciones seleccionando aleatoriamente cada combinación de hiperparámetros
random_search.fit(X_train, y_train)

In [ ]:
## Vemos cuál ha sido el mejor modelo y su R^2
print(f"Mejor score: \n{random_search.best_score_}\n")
print(f"Mejor modelo: \n{random_search.best_estimator_}")

In [40]:
## Guardamos el mejor modelo para usarlo
model = random_search.best_estimator_

In [ ]:
## Volvemos a validar con entrenamiento y test
print("Score training = {}".format(model.score(X_train, y_train)))
print("Score test = {}".format(model.score(X_test, y_test)))

## Hay sobreajuste!

In [42]:
## Calculamos la predicciones para poder graficarlas
y_train_est = model.predict(X_train)
y_test_est  = model.predict(X_test)

In [ ]:
mi = min(y_train_est.min(), y_test_est.min(), y_train.min(), y_test.min())
ma = max(y_train_est.max(), y_test_est.max(), y_train.max(), y_test.max())
aux = [mi, ma]

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(y_train, y_train_est, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (training)")
plt.ylabel("Precio estimado (training)")

plt.subplot(1,2,2)
plt.plot(y_test, y_test_est, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (test)")
plt.ylabel("Precio estimado (test)");

## Comparativa con modelo lineal

En la práctica, es bueno poder marcar modelos o reglas *baseline* con las que comparar el modelo que hemos entrenado y cuantificar cuánto de bueno es.

In [ ]:
## Instanciamos y entrenamos un modelo de regresión
model2 = LinearRegression()
model2.fit(X_train, y_train)

## Vemos R^2
print(f"Modelo de regresión lineal\n")
print("Score training = {}".format(model2.score(X_train, y_train)))
print("Score test = {}".format(model2.score(X_test, y_test)))

print(f"\nModelo de árbol de regresión\n")
print("Score training = {}".format(model.score(X_train, y_train)))
print("Score test = {}".format(model.score(X_test, y_test)))

In [45]:
## Calculamos predicciones
y_train_est2 = model2.predict(X_train)
y_test_est2  = model2.predict(X_test)

In [ ]:
mi = min(y_train_est2.min(), y_test_est2.min(), y_train.min(), y_test.min())
ma = max(y_train_est2.max(), y_test_est2.max(), y_train.max(), y_test.max())
aux = [mi, ma]

plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
plt.plot(y_train, y_train_est2, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (training)")
plt.ylabel("Precio estimado (training)")

plt.subplot(1,2,2)
plt.plot(y_test, y_test_est2, ".")
plt.plot(aux, aux) # diagonal
plt.xlabel("Precio real (test)")
plt.ylabel("Precio estimado (test)");